In [1]:
##THE HYBRID WITH SLIME AND IALS RESULTS ARE BETTER THAN ANY OTHER ALGORITHM TESTED BEFORE
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores, MergeTwoModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
stacked_train = combine_matrices(ICM=ICM, URM=URM_train)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13638 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13631 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import numpy as np 
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)
profile_length, profile_length.shape

(array([121, 427, 105, ..., 191, 132, 240], dtype=int32), (13650,))

In [5]:
block_size = int(len(profile_length)*0.05)
block_size

682

In [6]:
sorted_users = np.argsort(profile_length)
sorted_users

array([ 9021,  9328, 12441, ...,  1141,  8526,   317])

In [7]:
for group_id in range(0, 20):
    start_pos = group_id * block_size
    end_pos = min((group_id+1) * block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))

Group 0, #users in group 682, average p.len 29.00, median 30.0, min 4, max 49
Group 1, #users in group 682, average p.len 66.13, median 67.0, min 49, max 81
Group 2, #users in group 682, average p.len 94.60, median 94.0, min 81, max 107
Group 3, #users in group 682, average p.len 118.61, median 118.0, min 107, max 130
Group 4, #users in group 682, average p.len 139.67, median 140.0, min 130, max 149
Group 5, #users in group 682, average p.len 158.63, median 158.5, min 149, max 168
Group 6, #users in group 682, average p.len 176.72, median 176.0, min 168, max 186
Group 7, #users in group 682, average p.len 196.37, median 196.0, min 186, max 206
Group 8, #users in group 682, average p.len 216.51, median 216.0, min 206, max 226
Group 9, #users in group 682, average p.len 236.23, median 236.0, min 227, max 245
Group 10, #users in group 682, average p.len 255.18, median 255.0, min 245, max 265
Group 11, #users in group 682, average p.len 276.60, median 276.0, min 265, max 288
Group 12, #use

In [8]:
##CREATION OF DIFFERENT RECOMMENDERS TUNED

### RP3Beta Recommender
RP3 = RP3betaRecommender(URM_train=URM_train)
RP3.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

RP3betaRecommender: URM Detected 1 ( 0.0%) items with no interactions.


In [ ]:
##SLIMELASTICNET Recommender
SLIME = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=stacked_train)
SLIME.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

 12%|████▌                                 | 2184/18059 [00:57<04:38, 57.04it/s]

In [ ]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=stacked_train)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

In [ ]:
##RP3HYBRIDSLIME_IALS
similarity_slime = SLIME.W_sparse
similarity_rp3 = RP3.W_sparse

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = stacked_train,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )
recommenders =[similarity_hybrid,IALS_recommender]
new_hybrid = MergeTwoModelsByScores(stacked_train, recommenders)
new_hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [ ]:
IALS_RP3 = MergeTwoModelsByScores(URM_train =stacked_train, recommenders= [IALS_recommender, RP32])
IALS_RP3.fit(alpha=0.5165217586951893,beta=0.8835185422119769)

In [ ]:
IALS_RP3_SLIME = MergeTwoModelsByScores(URM_train =stacked_train, recommenders= [IALS_RP3, SLIME2])
IALS_RP3_SLIME.fit(alpha=0.23588582912644468,beta=1.0)

In [ ]:
##PURE_SVD
PURE_SVD = PureSVDItemRecommender(stacked_train)
PURE_SVD.fit(num_factors=23,topK=500)

In [ ]:
SLIME_PURESVD2 = MergeTwoModelsByScores(URM_train =stacked_train, recommenders= [SLIME,PURE_SVD])
SLIME_PURESVD2.fit(alpha=0.790241473254688,beta=0.043735415548593955)

In [ ]:
##P3ALPHA Recommender
P3Alpha_recommender = P3alphaRecommender(URM_train=stacked_train)
P3Alpha_recommender.fit(topK=46,alpha=0.7723235971713814, implicit=True, normalize_similarity=False)

In [ ]:
##RP3_P3A
similarity_rp3= RP3.W_sparse
similarity_p3a=P3Alpha_recommender.W_sparse
RP3_P3A = ItemKNNSimilarityHybridRecommender(URM_train = stacked_train, Similarity_1 = similarity_rp3, Similarity_2=similarity_p3a )
RP3_P3A.fit(alpha=0.9849405466567409)

In [ ]:
SLIME_PURESVD_RP3_P3A = MergeTwoModelsByScores(URM_train =stacked_train, recommenders= [SLIME_PURESVD,RP3_P3A])
SLIME_PURESVD_RP3_P3A.fit(alpha=0.6056872631076499,beta=0.12958301894152272)

In [ ]:
MAP_recommender_per_group = {}

recommenders= {"HYBRID_new": new_hybrid,
               "SLIME_PURESVD_RP3_P3A":SLIME_PURESVD_RP3_P3A,
               "SLIME_PURESVD2":SLIME_PURESVD2,
               "SLIME_PURESVD_RP32": SLIME_PURESVD_RP32,
               "IALS_SLIME_RP32":IALS_RP3_SLIME
}
    
recommender_object_dict = {}

for label, recommender in recommenders.items():
    recommender_object = recommender
    recommender_object_dict[label] = recommender_object



In [ ]:
cutoff = 10

for group_id in range(0, 20):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))
    
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    
    for label, recommender in recommender_object_dict.items():
        result_df, _ = evaluator_test.evaluateRecommender(recommender)
        if label in MAP_recommender_per_group:
            MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
        else:
            MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

_ = plt.figure(figsize=(16, 9))
for label, recommender in recommender_object_dict.items():
    results = MAP_recommender_per_group[label]
    plt.scatter(x=np.arange(0,len(results)), y=results, label=label)
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()